In [1]:
from behavior_python.detection.wheelDetectionSession import WheelDetectionSession
from behavior_python.plotters.detection.wheelDetectionTrialPlotter import DetectionTrialPlotter

import glob
import polars as pl

import matplotlib.pyplot as plt
from ipywidgets import Layout, Dropdown,interact,FloatRangeSlider
from IPython.display import display

In [ ]:
animalid = 'KC144'
area = '*'
glob.glob(f'J:\\presentation\\*_{animalid}_detect_opto*_{area}*')

In [2]:
exp_name = '230831_KC145_detect_opto120_AL__no_cam_KC' 
w = WheelDetectionSession(exp_name,load_flag=True)


[23-10-27 16:34:31] - Found session rawdata at J:\presentation
[23-10-27 16:34:31] -  : Started analysis of 230831_KC145_detect_opto120_AL__no_cam_KC
[23-10-27 16:34:31] -  : Found saved data: J:\analysis\230831_KC145_detect_opto120_AL__no_cam_KC\sessionData.parquet
[23-10-27 16:34:31] -  : Loading from J:\analysis\230831_KC145_detect_opto120_AL__no_cam_KC
[23-10-27 16:34:31] - Loaded all data : 0.136s
[23-10-27 16:34:31] - Setting outcome to state
[23-10-27 16:34:31] -  : Done! t=0.14 s


In [3]:
trial_plotter = DetectionTrialPlotter(w.data.data)
outcome_dict = {'Hit' : 1,
                'Miss' : 0,
                'Early' : -1,
                'All' : None
                }

outcome_selector =Dropdown(options=list(outcome_dict.keys()), 
                                    value='All',
                                    description='Trial Outcome')

trials = w.data.data.get_column('trial_no').to_list()

trial_selector = Dropdown(options=trials,
                                   value=1,
                                   description='Trial No')

range_slider = FloatRangeSlider(
    value=[-100, 1000],
    min=-5000,
    max=5000,
    step=10,
    description='Plot Range:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

def update_trial_list(*args):
    selected_outcome = outcome_selector.value
    o = outcome_dict[selected_outcome]
    if not o is None:
        filt_df = w.data.data.filter(pl.col('outcome')==o)
        filt_trials = filt_df.get_column('trial_no').to_list()
    else:
        filt_trials = w.data.data.get_column('trial_no').to_list()
    
        trial_selector.options = filt_trials
outcome_selector.observe(update_trial_list, 'change')
    
@interact(outcome=outcome_selector,trial_no=trial_selector,t_lim=range_slider)
def plot_trial(outcome,trial_no,t_lim):
    # filt_trials = trials
    # if outcome is not None:
    #     filt_df = w.data.data.filter(pl.col('outcome')==outcome_dict[outcome])
    #     filt_trials = filt_df.get_column('trial_no').to_list()
    # print(f'Number of trials: {len(filt_trials)}')
    trial_plotter.plot(trial_no=trial_no,t_lim=t_lim)

interactive(children=(Dropdown(description='Trial Outcome', index=3, options=('Hit', 'Miss', 'Early', 'All'), …